# Load Data

In [ ]:
import os
import pandas as pd
import platform
import numpy as np
from ta.volatility import AverageTrueRange
from ta.momentum    import RSIIndicator
from ta.trend       import MACD, EMAIndicator, ADXIndicator
from ta.volatility  import AverageTrueRange, BollingerBands
from itertools import product
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, root_mean_squared_error, mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from datetime import timedelta
from collections import defaultdict
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import label_binarize
from sklearn.inspection import permutation_importance
import optuna
import joblib
import json
import seaborn as sns
import shap
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message=".*There are no meaningful features.*", category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.INFO)

# === Load Data ===
folder_path = "./../data/"
column_names = ['datetime', 'open', 'high', 'low', 'close', 'volume']
df_list = []

system = platform.system()
# Set emoji-compatible font based on OS
if system == 'Windows':
    plt.rcParams['font.family'] = 'Segoe UI Emoji'
elif system == 'Linux':
    plt.rcParams['font.family'] = 'Noto Color Emoji'  # if installed

for filename in os.listdir(folder_path):
    if filename.endswith(('.csv', '.txt')):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep=';', header=None, names=column_names)
        df['source_file'] = filename
        df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df['datetime'] = pd.to_datetime(df['datetime'], utc=True).dt.tz_convert('America/New_York')

df = df.drop_duplicates(subset='datetime', keep='first').reset_index(drop=True)
df = df.sort_values('datetime').reset_index(drop=True)
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

# Base time features
df['hour'] = df['datetime'].dt.hour + df['datetime'].dt.minute / 60
df['minute'] = df['datetime'].dt.minute
df['day_of_week'] = df['datetime'].dt.dayofweek  # 0 = Monday

# Custom session flags (adjust if needed)       # Regular Trading Hours
df['is_premarket'] = df['hour'].between(7, 9.5)
df['is_lunch'] = df['hour'].between(11.5, 13.5)
df['is_postmarket'] = df['hour'].between(15.5, 20)
df['is_after_hours'] = df['hour'].between(20, 23.5)


# Initialize features or indicators

In [ ]:
# === ATR + Volatility Context ===
df['atr_5'] = AverageTrueRange(df['high'], df['low'], df['close'], window=5).average_true_range()
df['atr_pct'] = df['atr_5'] / df['close']  # normalized ATR

df['candle_range'] = (df['high'] - df['low'])

# === Choppiness Index (Trend vs. Noise) ===
def choppiness_index(high, low, close, length=14):
    tr = AverageTrueRange(high=high, low=low, close=close, window=length).average_true_range()
    atr_sum = tr.rolling(length).sum()
    high_max = high.rolling(length).max()
    low_min = low.rolling(length).min()
    return 100 * np.log10(atr_sum / (high_max - low_min)) / np.log10(length)

df['chop_index'] = choppiness_index(df['high'], df['low'], df['close'], length=14)

# === EMA-Based Features ===
df['ema_9'] = df['close'].ewm(span=9, adjust=False).mean()
df['ema_dist'] = (df['close'] - df['ema_9'])

# === Time Features ===
df['hour'] = df['datetime'].dt.hour
df['is_afternoon'] = (df['hour'] >= 12).astype(int)
df['is_morning'] = (df['hour'] < 12).astype(int)

# === Volume-based features ===
df['volume'] = df['volume'].astype(float)
df['volume_delta'] = df['volume'].diff()
df['volume_delta_ema'] = df['volume_delta'].ewm(span=10, adjust=False).mean()

# === Bollinger Band metrics ===
bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bollinger_width'] = (bb.bollinger_hband() - bb.bollinger_lband())

# === EMA slope ===
ema = EMAIndicator(close=df['close'], window=21)
df['ema_21'] = ema.ema_indicator()
df['ema_slope'] = df['ema_21'].diff()

# === VWAP (Volume Weighted Average Price)
vwap_numerator = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum()
vwap_denominator = df['volume'].cumsum()
df['vwap'] = vwap_numerator / (vwap_denominator + 1e-9)

# === Candlestick features
df['body_pct'] = np.abs(df['close'] - df['open']) / (df['high'] - df['low'] + 1e-9)
df['price_vs_vwap'] = df['close'] - df['vwap']

# # === Strategy Setup ===
param_grid_strategy = {
    'SL_ATR_MULT': [1.0, 1.5, 0.5],
    'TP_ATR_MULT': [2.0, 3.0, 4.0],
    'TRAIL_START_MULT': [0.5, 1.0],
    'TRAIL_STOP_MULT': [0.5, 1.0],
    'TICK_VALUE': [5], 
}

keys, values = zip(*param_grid_strategy.items())
combinations = [dict(zip(keys, v)) for v in product(*values)]

features = [
    'atr_5', 'atr_pct', 'ema_dist',
    'chop_index',
    'volume', 'volume_delta_ema',
    'bollinger_width',
    'ema_slope', 'body_pct',
    'price_vs_vwap', 'vwap',
]


# Load CSV of high-impact events
# Load news data
# news_df = pd.read_csv("high_impact_us_news.csv", parse_dates=["datetime"])

# # Localize datetime to New York timezone
# news_df["start_time"] = pd.to_datetime(news_df["datetime"]).dt.tz_localize("America/New_York")

# # Assume each news event blocks 30 minutes after its start
# news_df["end_time"] = news_df["start_time"] + pd.Timedelta(minutes=15)

# # Now create list of blackout windows
# news_windows = list(zip(news_df["start_time"], news_df["end_time"]))

def avoid_news(row):
    ts = row["datetime"]
    return any(start <= ts <= end for (start, end) in news_windows)

def avoid_hour_18_19(row):
    """
    Avoid trading in the first hour of the session (18:00 to 19:00 inclusive).
    """
    if not pd.api.types.is_datetime64_any_dtype(row['datetime']):
        return False
    hour = row['datetime'].hour
    return hour == 18

avoid_funcs = {
    #'avoid_hour_18_19': avoid_hour_18_19
    #'news_window': avoid_news,
}

def session_key(ts: pd.Timestamp) -> pd.Timestamp:
    # shift back 18 h, then floor to midnight to get a unique session “date”
    return (ts - timedelta(hours=18)).normalize()

def is_same_session(start_time: pd.Timestamp, end_time: pd.Timestamp) -> bool:
    return session_key(start_time) == session_key(end_time)

# Declare Combo function for serialization

In [ ]:
def evaluate_regression_combo(
    X_test, preds, labeled, df,
    avoid_funcs,
    SL_ATR_MULT, TP_ATR_MULT, TRAIL_START_MULT, TRAIL_STOP_MULT, TICK_VALUE,
    is_same_session,
    long_thresh=0.003,
    short_thresh=-0.003,
    base_contracts=1,
    max_contracts=5,
    skip_weak_conf=False,
    weak_conf_zscore=0.2
):
    temp_trades_data = []
    skipped_trades = 0
    avoid_hits = defaultdict(int)
    long_trades = 0
    short_trades = 0

    i = 0
    X_test_idx = X_test.index.to_list()
    preds_array = np.array(preds)

    # === Calculate z-score confidence ===
    zscores = (preds_array - preds_array.mean()) / (preds_array.std() + 1e-9)
    conf_scores = np.clip(np.abs(zscores), 0, 2.0)
    position_sizes = base_contracts + (max_contracts - base_contracts) * (conf_scores / 2.0)
    position_sizes = np.round(position_sizes, 2)


    for i, idx in enumerate(X_test_idx):
        #idx = X_test_idx[i]
        row = labeled.loc[idx]

        if idx + 1 >= len(df):
            skipped_trades += 1
            continue

        pred_return = preds_array[i]
        conf = conf_scores[i]
        size = position_sizes[i]

        # Skip weak confidence signals if enabled
        if skip_weak_conf and conf < weak_conf_zscore:
            skipped_trades += 1
            continue

        if pred_return >= long_thresh:
            side = 'long'
            long_trades += 1
        elif pred_return <= short_thresh:
            side = 'short'
            short_trades += 1
        else:
            i += 1
            continue  # skip neutral signals

        # Trade filters
        skip_trade = False
        for name, f in avoid_funcs.items():
            try:
                if f(row):
                    avoid_hits[name] += 1
                    skip_trade = True
            except:
                continue
        if skip_trade:
            skipped_trades += 1
            i += 1
            continue

        # --- Trade Simulation ---
        entry_price = df.loc[idx + 1, 'open']
        entry_time = df.loc[idx + 1, 'datetime']
        atr = row['atr_5']

        # Stop Loss (fixed volatility-based)
        sl_price = entry_price - SL_ATR_MULT * atr if side == 'long' else entry_price + SL_ATR_MULT * atr

        # Take Profit (dynamic, from model prediction, clipped)
        expected_move = abs(pred_return) * entry_price
        min_tp = 0.001 * entry_price  # minimum 0.1% move
        max_tp = TP_ATR_MULT * atr
        tp_move = np.clip(expected_move, min_tp, max_tp)
        tp_price = entry_price + tp_move if side == 'long' else entry_price - tp_move

        # Trailing logic
        trail_trigger = entry_price + TRAIL_START_MULT * atr if side == 'long' else entry_price - TRAIL_START_MULT * atr
        trail_stop = None

        max_price, min_price = entry_price, entry_price
        exit_price, exit_time = None, None

        fwd_idx = idx + 1
        while fwd_idx < len(df):
            fwd_row = df.loc[fwd_idx]
            max_price = max(max_price, fwd_row['high'])
            min_price = min(min_price, fwd_row['low'])

            if (side == 'long' and fwd_row['low'] <= sl_price) or (side == 'short' and fwd_row['high'] >= sl_price):
                exit_price = sl_price
                exit_time = fwd_row['datetime']
                break

            if (side == 'long' and fwd_row['high'] >= tp_price) or (side == 'short' and fwd_row['low'] <= tp_price):
                exit_price = tp_price
                exit_time = fwd_row['datetime']
                break

            if side == 'long' and fwd_row['high'] >= trail_trigger:
                trail_stop = fwd_row['close'] - TRAIL_STOP_MULT * atr
            if side == 'short' and fwd_row['low'] <= trail_trigger:
                trail_stop = fwd_row['close'] + TRAIL_STOP_MULT * atr

            if trail_stop:
                if (side == 'long' and fwd_row['low'] <= trail_stop) or (side == 'short' and fwd_row['high'] >= trail_stop):
                    exit_price = trail_stop
                    exit_time = fwd_row['datetime']
                    break

            fwd_idx += 1

        if exit_price is None:
            exit_price = df.loc[len(df) - 1, 'close']
            exit_time = df.loc[len(df) - 1, 'datetime']

        if not is_same_session(entry_time, exit_time):
            i += 1
            continue

        GROSS_PNL = (exit_price - entry_price) * TICK_VALUE * size if side == 'long' else (entry_price - exit_price) * TICK_VALUE * size
        COMMISSION = 3.98 * size
        pnl = GROSS_PNL - COMMISSION

        mfe = max_price - entry_price if side == 'long' else entry_price - min_price
        mae = entry_price - min_price if side == 'long' else max_price - entry_price

        temp_trades_data.append({
            'entry_time': entry_time,
            'exit_time': exit_time,
            'side': side,
            'entry_price': entry_price,
            'exit_price': exit_price,
            'pnl': pnl,
            'mfe': mfe,
            'mae': mae,
            'gross_pnl': GROSS_PNL,
            'pred_return': pred_return,
            'confidence': conf,
            'position_size': size,
        })

        while i < len(X_test_idx) and labeled.loc[X_test_idx[i]]['datetime'] <= exit_time:
            i += 1
        continue

    # === Metrics ===
    results = pd.DataFrame(temp_trades_data)
    pnl_total = results['pnl'].sum() if not results.empty else 0
    trades = len(results)
    win_rate = (results['pnl'] > 0).mean() if not results.empty else 0
    expectancy = results['pnl'].mean() if not results.empty else 0
    profit_factor = results[results['pnl'] > 0]['pnl'].sum() / abs(results[results['pnl'] < 0]['pnl'].sum()) if not results.empty and (results['pnl'] < 0).any() else np.nan
    sharpe = results['pnl'].mean() / (results['pnl'].std() + 1e-9) * np.sqrt(trades) if trades > 1 else 0

    return {
        'pnl': pnl_total,
        'trades': trades,
        'win_rate': win_rate,
        'expectancy': expectancy,
        'profit_factor': profit_factor,
        'sharpe': sharpe,
        'long_trades': long_trades,
        'short_trades': short_trades,
        'avoid_hits': dict(avoid_hits),
        'threshold': long_thresh,
        'results': results
    }

# Cleanup

In [ ]:
def compute_future_return_labels(df: pd.DataFrame, lookahead: int, is_same_session_fn) -> pd.DataFrame:
    """
    Computes future return (regression label) and trade direction for a given lookahead period.

    Parameters:
    - df: DataFrame with at least ['datetime', 'close']
    - lookahead: How many bars ahead to evaluate performance
    - is_same_session_fn: Function that checks if two datetimes are in the same session

    Returns:
    - df_labeled: DataFrame with ['future_return', 'trade_dir'] added
    """
    future_returns = []
    trade_dirs = []

    for idx in range(len(df) - lookahead):
        entry_price = df['close'].iloc[idx]
        future_price = df['close'].iloc[idx + lookahead]

        if pd.isna(entry_price) or pd.isna(future_price):
            future_returns.append(np.nan)
            trade_dirs.append(None)
            continue

        future_return = (future_price / entry_price) - 1
        future_returns.append(future_return)
        trade_dirs.append('long' if future_return > 0 else 'short')

    # Align output with original df
    df_labeled = df.iloc[:len(future_returns)].copy()
    df_labeled['future_return'] = future_returns
    df_labeled['trade_dir'] = trade_dirs

    # Drop NaNs
    df_labeled = df_labeled.dropna(subset=['future_return'])

    return df_labeled 

In [ ]:
def compute_tp_sl_labels_with_strength(df: pd.DataFrame, lookahead: int, is_same_session_fn,
    sl_atr_mult: float = 1.0,
    tp_atr_mult: float = 2.0,
    strong_tp_mult: float = 4.0  # e.g., 2x ATR SL (twice as far as SL)
) -> pd.DataFrame:
    labels = []
    valid_idxs = []

    for i in range(len(df) - lookahead):
        entry_time = df['datetime'].iloc[i]
        end_time = df['datetime'].iloc[i + lookahead]

        if not is_same_session_fn(entry_time, end_time):
            continue

        entry_price = df['close'].iloc[i]
        atr = df['atr_5'].iloc[i]

        sl = entry_price - sl_atr_mult * atr
        tp = entry_price + tp_atr_mult * atr
        strong_tp = entry_price + strong_tp_mult * atr

        future = df.iloc[i+1:i+1+lookahead]
        label = 0

        for _, row in future.iterrows():
            if row['low'] <= sl:
                label = -1
                break
            if row['high'] >= strong_tp:
                label = 2
                break
            if row['high'] >= tp:
                label = 1
                break

        labels.append(label)
        valid_idxs.append(i)

    df_out = df.iloc[valid_idxs].copy()
    df_out['tp_sl_label'] = labels
    return df_out


In [ ]:
lookahead_values = [5, 15]

def label_and_save(lookahead):
    df_session = df.copy()
    print(f"Initial rows: {len(df_session)}")

    df_labeled = compute_future_return_labels(
        df_session,
        lookahead=lookahead,
        is_same_session_fn=is_same_session
    )
    print(f"➤ Rows after future_return: {len(df_labeled)} | Dropped: {len(df_session) - len(df_labeled)}")


    df_exec = compute_tp_sl_labels_with_strength(df_session, lookahead, is_same_session_fn=is_same_session)
    print(f"➤ Rows after tp_sl_label: {len(df_exec)} | Dropped: {len(df_session) - len(df_exec)}")


    df_combined = df_labeled.merge(df_exec[['datetime', 'tp_sl_label']], on='datetime', how='left')
    print(f"➤ Rows after merging: {len(df_combined)}")
    print(f"➤ tp_sl_label NaNs after merge: {df_combined['tp_sl_label'].isna().sum()}")

    rows_before_final = len(df_combined)
    df_final = df_combined.dropna(subset=['future_return', 'tp_sl_label'] + features)
    print(f"➤ Rows after final drop: {len(df_final)} | Dropped: {rows_before_final - len(df_final)}")

    # Step 5: Save parquet
    df_final.to_parquet(f"labeled_data_{lookahead}_session_less.parquet")
    print(f"✅ Saved labeled_data_{lookahead}_session_less.parquet with {len(df_final)} rows")

for lookahead in lookahead_values:
    fname = f"labeled_data_{lookahead}_session_less.parquet"
    if os.path.exists(fname):
        print(f"⏭️ File {fname} already exists. Skipping...")
        continue
    print(f"📦 Labeling {fname}...")
    label_and_save(lookahead)


# Train

##### Real Training

In [ ]:
def check_overfit(model, X_tr, X_te, y_tr, y_te):
    train_preds = model.predict(X_tr)
    test_preds = model.predict(X_te)
    train_mse = mean_squared_error(y_tr, train_preds)
    test_mse = mean_squared_error(y_te, test_preds)
    ratio = test_mse / train_mse if train_mse != 0 else float('inf')

    print(f"\n📉 Overfitting check:")
    print(f"Train MSE: {train_mse:.8f}")
    print(f"Test MSE: {test_mse:.8f}")
    print(f"Overfit ratio (Test / Train): {ratio:.2f}")
    if ratio > 1.5:
        print("⚠️ Potential overfitting detected.")
    elif ratio < 0.7:
        print("⚠️ Possibly underfitting (too simple).")
    else:
        print("✅ Generalization looks reasonable.")

In [ ]:
def generate_oof_predictions(models, X, y, n_splits=3):
    oof_preds = np.zeros((X.shape[0], len(models)))
    tscv = TimeSeriesSplit(n_splits=n_splits)

    for i, model in enumerate(models):
        for train_idx, val_idx in tscv.split(X):
            model.fit(X.iloc[train_idx], y.iloc[train_idx])
            oof_preds[val_idx, i] = model.predict(X.iloc[val_idx])
    
    return pd.DataFrame(oof_preds, index=y.index)  # ✅ FIX is here


In [ ]:
def run_lookahead_for_session(LOOKAHEAD):
    labeled = pd.read_parquet(f"labeled_data_{LOOKAHEAD}_session_less.parquet")
    labeled = labeled.replace([np.inf, -np.inf], np.nan)
    labeled = labeled.dropna(subset=features + ['future_return', 'tp_sl_label'])

    cutoff_date = pd.Timestamp("2025-01-01", tz="America/New_York")
    train = labeled[labeled['datetime'] < cutoff_date]
    test = labeled[labeled['datetime'] >= cutoff_date]

    y_train_class = train['tp_sl_label']
    y_test_class = test['tp_sl_label']

    X_train_full, y_train_reg = train[features], train['future_return']
    X_test_full, y_test_reg = test[features], test['future_return']

    print(f"Train range: {train['datetime'].min()} to {train['datetime'].max()} | Rows: {len(train)}")
    print(f"Test range: {test['datetime'].min()} to {test['datetime'].max()} | Rows: {len(test)}")

    ###########################
    ###### Classifiers ########
    ###########################

    def tune_catboost_classifier(X_train, y_train):
        def objective(trial):
            params = {
                'iterations': 1000,
                'depth': trial.suggest_int('depth', 4, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 5.0),
                'bootstrap_type': 'Bayesian',
                'random_strength': trial.suggest_float('random_strength', 1.0, 5.0),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100)
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = CatBoostClassifier(**params, random_state=42)
                model.fit(X_tr, y_tr, eval_set=(X_val, y_val))

                probs = model.predict_proba(X_val)
                y_val_bin = label_binarize(y_val, classes=model.classes_)

                score = roc_auc_score(y_val_bin, probs, multi_class='ovr', average='macro')
                scores.append(score)

            return -np.mean(scores)

        study = optuna.create_study(
            direction="minimize",
            study_name="catboost_classifier_opt",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage=f"sqlite:///catboost_classifier_opt_study{LOOKAHEAD}_session_less.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=25)
        return study.best_params
    
    def tune_lgbm_classifier(X_train, y_train):
        def objective(trial):
            params = {
                "n_estimators": 1000,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 8),
                "num_leaves": trial.suggest_int("num_leaves", 31, 256),
                "subsample": trial.suggest_float("subsample", 0.7, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
                "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0)
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = LGBMClassifier(**params, random_state=42)
                model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])

                probs = model.predict_proba(X_val)
                y_val_bin = label_binarize(y_val, classes=model.classes_)

                score = roc_auc_score(y_val_bin, probs, multi_class='ovr', average='macro')
                scores.append(score)

            return -np.mean(scores)

        study = optuna.create_study(
            direction="minimize",
            study_name="lgbm_classifier_opt",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage=f"sqlite:///lgbm_classifier_opt_study{LOOKAHEAD}_session_less.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=5)
        return study.best_params

    def tune_xgb_classifier(X_train, y_train):
        def objective(trial):
            params = {
                "n_estimators": 1000,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 8),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
                "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0)
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = XGBClassifier(**params, eval_metric="auc", random_state=42, use_label_encoder=False)
                model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])
                preds = model.predict_proba(X_val)[:, 1]
                score = roc_auc_score(y_val, preds)
                scores.append(score)

            return -np.mean(scores)

        study = optuna.create_study(
            direction="minimize",
            study_name="xgb_classifier_opt",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage=f"sqlite:///xgb_classifier_opt_study{LOOKAHEAD}_session_less.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=25)
        return study.best_params

    def tune_rf_classifier(X_train, y_train):
        def objective(trial):
            params = {
                'n_estimators': 500,
                'max_depth': trial.suggest_int('max_depth', 4, 20),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = RandomForestClassifier(**params, random_state=42)
                model.fit(X_tr, y_tr)
                preds = model.predict_proba(X_val)[:, 1]
                score = roc_auc_score(y_val, preds)
                scores.append(score)

            return -np.mean(scores)

        study = optuna.create_study(
            direction="minimize",
            study_name="rf_classifier_opt",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage=f"sqlite:///rf_classifier_opt_study{LOOKAHEAD}_session_less.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=25)
        return study.best_params

    def tune_logreg_classifier(X_train, y_train):
        def objective(trial):
            params = {
                "C": trial.suggest_float("C", 1e-3, 10.0, log=True),
                "penalty": "l2",
                "solver": "lbfgs",
                "max_iter": 1000
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = LogisticRegression(**params, random_state=42)
                model.fit(X_tr, y_tr)
                preds = model.predict_proba(X_val)[:, 1]
                score = roc_auc_score(y_val, preds)
                scores.append(score)

            return -np.mean(scores)

        study = optuna.create_study(
            direction="minimize",
            study_name="logreg_classifier_opt",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage=f"sqlite:///logreg_classifier_opt_study{LOOKAHEAD}_session_less.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=25)
        return study.best_params

    def tune_meta_classifier(X_meta, y_meta):
        """
        Optuna-tune Logistic Regression meta-classifier for stacking.
        Works on out-of-fold predicted probabilities from base classifiers.
        """
        def objective(trial):
            params = {
                "C": trial.suggest_float("C", 1e-4, 100.0, log=True),
                "penalty": trial.suggest_categorical("penalty", ["l2"]),
                "solver": "lbfgs",
                "max_iter": 1000
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_meta):
                X_tr, X_val = X_meta.iloc[train_idx], X_meta.iloc[val_idx]
                y_tr, y_val = y_meta.iloc[train_idx], y_meta.iloc[val_idx]

                model = LogisticRegression(**params, random_state=42)
                model.fit(X_tr, y_tr)
                preds = model.predict(X_val)
                f1 = f1_score(y_val, preds, average="macro")
                scores.append(f1)

            return np.mean(scores)

        study = optuna.create_study(
            direction="maximize",
            study_name="meta_logreg_stack_cls",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage="sqlite:///meta_logreg_stack_cls.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=50)
        return study.best_params

    ###########################
    ###### Regressors #########
    ###########################

    def tune_lightgbm(X_train, y_train):
        def objective(trial):
            params = {
                "n_estimators": 2000,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 8),
                "num_leaves": trial.suggest_int("num_leaves", 31, 256),
                "min_child_samples": trial.suggest_int("min_child_samples", 1, 30),
                "subsample": trial.suggest_float("subsample", 0.7, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
                "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
                "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.01),
                "force_col_wise": trial.suggest_categorical("force_col_wise", [True, False])
            }
            tscv = TimeSeriesSplit(n_splits=3)
            scores = []
            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = LGBMRegressor(**params, random_state=42, n_jobs=-5)
                model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_val, y_val)],
                    eval_metric="rmse"
                )
                preds = model.predict(X_val)
                rmse = root_mean_squared_error(y_val, preds)
                scores.append(rmse)
            return np.mean(scores)

        study = optuna.create_study(
            direction="minimize",
            study_name="lgbm_opt",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=50, reduction_factor=4),
            storage=f"sqlite:///lgbm_opt_study{LOOKAHEAD}_session_less.db",
            load_if_exists=True
        )
        study.optimize(objective, n_trials=5)
        return study.best_params

    def tune_catboost(X_train, y_train):
        def objective(trial):
            params = {
                'iterations': 2000,
                'depth': trial.suggest_int('depth', 4, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
                'loss_function': 'RMSE',
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3.0, 10.0),
                'random_strength': trial.suggest_float('random_strength', 1.0, 5.0),
                'bootstrap_type': 'Bayesian',
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                model = CatBoostRegressor(**params, random_state=42)
                model.fit(
                    X_tr, y_tr,
                    eval_set=(X_val, y_val),
                    use_best_model=True,
                    verbose=False,
                    early_stopping_rounds=30
                )
                preds = model.predict(X_val)
                rmse = root_mean_squared_error(y_val, preds)
                scores.append(rmse)

            return np.mean(scores)

        study = optuna.create_study(
            direction='minimize',
            study_name='catboost_opt',
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=50, reduction_factor=4),
            storage=f'sqlite:///catboost_opt_study{LOOKAHEAD}_session_less.db',
            load_if_exists=True
        )
        study.optimize(objective, n_trials=50)
        return study.best_params

    def tune_meta_regressor(X_meta, y_meta):
        """
        Optuna-tune CatBoostRegressor as meta-learner for stacking.
        """
        def objective(trial):
            params = {
                'iterations': 2000,
                'depth': trial.suggest_int('depth', 4, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
                'loss_function': 'RMSE',
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3.0, 10.0),
                'random_strength': trial.suggest_float('random_strength', 1.0, 5.0),
                'bootstrap_type': 'Bayesian',
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
            }

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []

            for train_idx, val_idx in tscv.split(X_meta):
                X_tr, X_val = X_meta.iloc[train_idx], X_meta.iloc[val_idx]
                y_tr, y_val = y_meta.iloc[train_idx], y_meta.iloc[val_idx]

                model = CatBoostRegressor(**params, random_state=42)
                model.fit(
                    X_tr, y_tr,
                    eval_set=(X_val, y_val),
                    use_best_model=True,
                    verbose=False,
                    early_stopping_rounds=30
                )
                preds = model.predict(X_val)
                rmse = root_mean_squared_error(y_val, preds)
                scores.append(rmse)

            return np.mean(scores)

        study = optuna.create_study(
            direction='minimize',
            study_name='meta_catboost_stack',
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
            storage=f'sqlite:///meta_catboost_stack_{lookahead}_session_less.db',
            load_if_exists=True
        )
        study.optimize(objective, n_trials=50)
        return study.best_params
    
    ####### Shap Plotting
    ####### REGRESSION #######
    X_shap_train, X_shap_val, y_shap_train, y_shap_val = train_test_split(
        X_train_full, y_train_reg, test_size=0.2, shuffle=False
    )
    
    print("Training model for SHAP analysis...")
    shap_model = LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42)
    shap_model.fit(X_shap_train, y_shap_train)

    print("Computing SHAP values...")
    explainer = shap.Explainer(shap_model, X_shap_train)
    shap_values = explainer(X_shap_val)

    # Calculate average SHAP importance per feature
    mean_shap = np.abs(shap_values.values).mean(axis=0)
    top_idx = np.argsort(mean_shap)[::-1][:25]
    top_features = X_shap_train.columns[top_idx].tolist()

    shap.summary_plot(shap_values, X_shap_val)
    shap.summary_plot(shap_values, X_shap_val, plot_type="bar")
    shap.plots.force(explainer(X_shap_val.iloc[0]), matplotlib=True)

    print("Top SHAP Features:", top_features)

    ###########################
    ###### Model Training #####
    ###########################
    X_train_combined = X_train_full[top_features]
    print(len(X_train_combined))
    X_test_combined = X_test_full[top_features]

    ################################################
    ####### Ensure index consistency
    ####### REGRESSION #######
    y_train_reg = y_train_reg.loc[X_train_combined.index]
    print(len(y_train_reg))
    y_test_reg = y_test_reg.loc[X_test_combined.index]
    ####### CLASSIFICATION #######
    y_train_class = y_train_class.loc[X_train_combined.index]
    y_test_class = y_test_class.loc[X_test_combined.index]

    ################################################
    ####### Tune models
    ####### REGRESSION #######
    lgbm_params         = tune_lightgbm(X_train_combined, y_train_reg)
    catboost_params     = tune_catboost(X_train_combined, y_train_reg)
    ####### CLASSIFICATION #######
    lgbm_params_class       = tune_lgbm_classifier(X_train_combined, y_train_class)
    catboost_params_class   = tune_catboost_classifier(X_train_combined, y_train_class)
    xgb_params_class       = tune_xgb_classifier(X_train_combined, y_train_class)
    rf_params_class       = tune_rf_classifier(X_train_combined, y_train_class)
    logreg_params_class       = tune_logreg_classifier(X_train_combined, y_train_class)

    ################################################
    ####### Train models
    ####### REGRESSION #######
    lgbm = LGBMRegressor(**lgbm_params, random_state=42, n_jobs=-5)
    catboost = CatBoostRegressor(**catboost_params, random_state=42, verbose=0)
    lgbm.fit(X_train_combined, y_train_reg)
    catboost.fit(X_train_combined, y_train_reg)
    ####### CLASSIFICATION #######
    lgbm_clf     = LGBMClassifier(**lgbm_params_class, random_state=42)
    catboost_clf = CatBoostClassifier(**catboost_params_class, random_state=42, verbose=0)
    xgb_clf      = XGBClassifier(**xgb_params_class, random_state=42, use_label_encoder=False, eval_metric='logloss')
    rf_clf       = RandomForestClassifier(**rf_params_class, random_state=42)
    logreg_clf   = LogisticRegression(**logreg_params_class, random_state=42, max_iter=1000)
    lgbm_clf.fit(X_train_combined, y_train_class)
    catboost_clf.fit(X_train_combined, y_train_class)
    xgb_clf.fit(X_train_combined, y_train_class)
    rf_clf.fit(X_train_combined, y_train_class)
    logreg_clf.fit(X_train_combined, y_train_class)
    
    ################################################
    ####### Base models selection
    X_meta_check = X_train_combined
    ####### REGRESSION #######
    base_models_reg = [
        lgbm,
       catboost
    ]
    filtered_models = []

    for i, model in enumerate(base_models_reg):
        try:
            preds = model.predict(X_meta_check)
            pred_std = np.std(preds)
            print(f"📊 Model {i} std: {pred_std:.6f}")
            if pred_std > 1e-4:
                filtered_models.append((f"model_{i}", model))
            else:
                print(f"⚠️ Skipping model_{i} due to low variance")
        except Exception as e:
            print(f"❌ Model {i} failed: {e}")
    ####### CLASSIFICATION #######
    base_models_class = [
        lgbm_clf,
       catboost_clf,
       xgb_clf,
       rf_clf,
       logreg_clf
    ]
    filtered_models_class = []

    for i, model in enumerate(base_models_class):
        try:
            preds = model.predict(X_meta_check)
            pred_std = np.std(preds)
            print(f"📊 Model {i} std: {pred_std:.6f}")
            if pred_std > 1e-4:
                filtered_models_class.append((f"model_{i}", model))
            else:
                print(f"⚠️ Skipping model_{i} due to low variance")
        except Exception as e:
            print(f"❌ Model {i} failed: {e}")

    ################################################
    ####### OOF Predicition
    ####### REGRESSION #######
    base_models_only = [model for name, model in filtered_models]
    base_models_preds_train = generate_oof_predictions(base_models_only, X_train_combined, y_train_reg)

    print("\n🔍 Checking variance in OOF base model predictions:")
    print(base_models_preds_train.describe())
    print("Std per model:\n", base_models_preds_train.std())
    ####### CLASSIFICATION #######
    base_models_only_class = [model for name, model in filtered_models_class]
    base_models_preds_train_class = generate_oof_predictions(base_models_only_class, X_train_combined, y_train_class)

    print("\n🔍 Checking variance in OOF base model predictions:")
    print(base_models_preds_train_class.describe())
    print("Std per model:\n", base_models_preds_train_class.std())

    ################################################
    ####### Meta Params and Training
    ####### REGRESSION #######
    meta_params = tune_meta_regressor(base_models_preds_train, y_train_reg)
    meta_model = CatBoostRegressor(**meta_params, random_state=42)

    stack = StackingRegressor(
        estimators=filtered_models,
        final_estimator=meta_model,
        n_jobs=-5
    )
    stack.fit(X_train_combined, y_train_reg)
    ####### CLASSIFICATION #######
    meta_params_class = tune_meta_classifier(base_models_preds_train, y_train_reg)
    meta_model_class = LogisticRegression(**meta_params_class, random_state=42)

    stack_class = StackingRegressor(
        estimators=filtered_models_class,
        final_estimator=meta_model_class,
        n_jobs=-5
    )
    stack_class.fit(X_train_combined, y_train_reg)

    ################################################
    ####### Evaluate Model
    def evaluate_model(name, model, Xtr, Xte, ytr, yte, scaled=False):
        train_preds = model.predict(Xtr)
        test_preds = model.predict(Xte)
        train_mse = mean_squared_error(ytr, train_preds)
        test_mse = mean_squared_error(yte, test_preds)
        overfit_ratio = test_mse / train_mse if train_mse != 0 else float('inf')

        print(f"\n📊 {name} Performance:")
        print(f"Train MSE: {train_mse:.8f}")
        print(f"Test MSE: {test_mse:.8f}")
        print(f"Overfit ratio (Test / Train): {overfit_ratio:.2f}")
        if overfit_ratio > 1.5:
            print("⚠️ Potential overfitting detected.")
        elif overfit_ratio < 0.7:
            print("⚠️ Possibly underfitting.")
        else:
            print("✅ Generalization looks reasonable.")
        return test_preds
    
    def evaluate_classifier(name, model, Xtr, Xte, ytr, yte):
        train_preds = model.predict(Xtr)
        test_preds = model.predict(Xte)

        train_acc = accuracy_score(ytr, train_preds)
        test_acc = accuracy_score(yte, test_preds)
        f1 = f1_score(yte, test_preds, average="macro")

        print(f"\n📊 {name} Classifier Performance:")
        print(f"Train Accuracy: {train_acc:.4f}")
        print(f"Test Accuracy: {test_acc:.4f}")
        print(f"F1 Score (Macro): {f1:.4f}")

        if test_acc / (train_acc + 1e-9) > 1.5:
            print("⚠️ Likely overfitting.")
        elif test_acc / (train_acc + 1e-9) < 0.7:
            print("⚠️ Possibly underfitting.")
        else:
            print("✅ Generalization looks reasonable.")

        return test_preds
    ####### REGRESSION #######
    preds_lgbm = evaluate_model("LightGBM", lgbm, X_train_combined, X_test_combined, y_train_reg, y_test_reg)
    preds_catboost = evaluate_model("CatBoostRegressor", catboost, X_train_combined, X_test_combined, y_train_reg, y_test_reg)
    preds_stack = evaluate_model("Stacking Ensemble", stack, X_train_combined, X_test_combined, y_train_reg, y_test_reg)
    ####### CLASSIFICATION #######
    # === Evaluate Classifiers ===
    preds_lgbm_clf     = evaluate_classifier("LightGBM Classifier", lgbm_clf, X_train_combined, X_test_combined, y_train_class, y_test_class)
    preds_catboost_clf = evaluate_classifier("CatBoost Classifier", catboost_clf, X_train_combined, X_test_combined, y_train_class, y_test_class)
    preds_xgb_clf      = evaluate_classifier("XGBoost Classifier", xgb_clf, X_train_combined, X_test_combined, y_train_class, y_test_class)
    preds_rf_clf       = evaluate_classifier("Random Forest Classifier", rf_clf, X_train_combined, X_test_combined, y_train_class, y_test_class)
    preds_logreg_clf   = evaluate_classifier("Logistic Regression Classifier", logreg_clf, X_train_combined, X_test_combined, y_train_class, y_test_class)
    preds_stack_clf    = evaluate_classifier("Stacking Ensemble Classifier", stack_class, X_train_combined, X_test_combined, y_train_class, y_test_class)


    ################################################
    ####### Target Distribution
    ####### CLASSIFICATION #######
    print("\n🔍 Classifier Insights")

    if hasattr(stack_class, "feature_importances_"):
        importances = pd.Series(stack_class.feature_importances_, index=X_train_combined.columns)
        importances = importances.sort_values(ascending=False)

        print("\n📊 Top 15 Feature Importances (LGBM):")
        print(importances.head(15))

        plt.figure(figsize=(10, 5))
        sns.barplot(x=importances.head(15).values, y=importances.head(15).index)
        plt.title("Top 15 Feature Importances (LGBMClassifier)")
        plt.xlabel("Importance")
        plt.tight_layout()
        plt.show()

    # === 2. Permutation Importance
    print("\n⏳ Calculating permutation importance (val set)...")
    perm_result = permutation_importance(
        stack_class, X_test_combined, y_test_class,
        scoring="accuracy", n_repeats=10, random_state=42
    )

    perm_importances = pd.Series(perm_result.importances_mean, index=X_test_combined.columns)
    perm_sorted = perm_importances.sort_values(ascending=False)

    print("\n📊 Top 15 Permutation Importances (Accuracy impact):")
    print(perm_sorted.head(15))

    plt.figure(figsize=(10, 5))
    sns.barplot(x=perm_sorted.head(15).values, y=perm_sorted.head(15).index)
    plt.title("Top 15 Permutation Importances (LGBMClassifier)")
    plt.xlabel("Mean Accuracy Impact")
    plt.tight_layout()
    plt.show()

    unique_classes, class_counts = np.unique(preds_stack_clf, return_counts=True)
    print("\n🔢 Predicted Class Distribution:")
    for cls, count in zip(unique_classes, class_counts):
        pct = 100 * count / len(preds_stack_clf)
        print(f"Class {cls}: {count} samples ({pct:.2f}%)")

    ################################################
    ####### Target Distribution
    ####### REGRESSION #######
    print("\n🔍 Target distribution:")
    print(y_train_reg.describe())
    ####### CLASSIFICATION #######
    print("\n🔍 Target distribution:")
    print(y_train_class.describe())
    
    ################################################
    ####### Choose final model
    ####### REGRESSION #######
    preds = stack.predict(X_test_combined)
    print("\n🔍 Checking prediction variance from stack model:")
    print(f"Min: {preds.min():.8f}")
    print(f"Max: {preds.max():.8f}")
    print(f"Mean: {preds.mean():.8f}")
    print(f"Std Dev: {preds.std():.8f}")
    print(f"First 5 Predictions: {preds[:5]}")

    all_results = []

    mae = mean_absolute_error(y_test_reg, preds)
    rmse = np.sqrt(mean_squared_error(y_test_reg, preds))
    r2 = r2_score(y_test_reg, preds)

    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    ####### CLASSIFICATION #######
    preds_class = stack_class.predict(X_test_combined)
    print("\n🔍 Checking prediction summary from classification stack model:")
    print(f"Class Distribution: {np.bincount(preds_class)}")
    print(f"First 5 Predictions: {preds_class[:5]}")

    accuracy = accuracy_score(y_test_class, preds_class)
    precision = precision_score(y_test_class, preds_class, average='weighted')
    recall = recall_score(y_test_class, preds_class, average='weighted')
    f1 = f1_score(y_test_class, preds_class, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (weighted): {precision:.4f}")
    print(f"Recall (weighted): {recall:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test_class, preds_class))

    return

    thresholds = [0.0005, 0.005]
    print(f"\n🔎 Predicted return range for LOOKAHEAD={LOOKAHEAD}: min={preds.min():.6f}, max={preds.max():.6f}")
    for params in combinations:
        for thresh in thresholds:
            results = evaluate_regression_combo(
                X_test=X_test_combined,
                preds=preds,
                labeled=labeled,
                df=df,
                avoid_funcs=avoid_funcs,
                SL_ATR_MULT=params['SL_ATR_MULT'],
                TP_ATR_MULT=params['TP_ATR_MULT'],
                TRAIL_START_MULT=params['TRAIL_START_MULT'],
                TRAIL_STOP_MULT=params['TRAIL_STOP_MULT'],
                TICK_VALUE=params['TICK_VALUE'],
                is_same_session=is_same_session,
                long_thresh=thresh,
                short_thresh=-thresh,

                # 🔥 NEW: enable confidence-based scaling and filtering
                base_contracts=1,
                max_contracts=5,
                skip_weak_conf=True,
                weak_conf_zscore=0.2
            )

            results['params'] = params
            all_results.append(results)

            print(f"\n\n🔍 Evaluating with params: {params}")

            print(
                f"\n✅ LOOKAHEAD={LOOKAHEAD} | Threshold={thresh}"
                f"\nPnL: ${results['pnl']:.2f}"
                f"\nTrades: {results['trades']}"
                f"\nWin Rate: {results['win_rate']:.2%}"
                f"\nExpectancy: {results['expectancy']:.2f}"
                f"\nProfit Factor: {results['profit_factor']:.2f}"
                f"\nSharpe Ratio: {results['sharpe']:.2f}"
                f"\nLong Trades: {results['long_trades']} | Short Trades: {results['short_trades']}"
            )

            print("Avoid Hits:")
            for name, count in results['avoid_hits'].items():
                print(f" - {name}: {count}")

            if not results['results'].empty and 'pnl' in results['results'].columns:
                print("\n🔢 Top 10 PnL trades:")
                print(results['results'].sort_values(by='pnl', ascending=False).head(10))

                print("\n🔻 Bottom 10 PnL trades:")
                print(results['results'].sort_values(by='pnl', ascending=True).head(10))
            else:
                print("\n⚠️ No trades executed, skipping PnL trade breakdown.")


    summary_df = pd.DataFrame([{
        'pnl': r['pnl'],
        'sharpe': r['sharpe'],
        'expectancy': r['expectancy'],
        'profit_factor': r['profit_factor'],
        'win_rate': r['win_rate'],
        'trades': r['trades'],
        **r['params']
    } for r in all_results])
    top = summary_df.sort_values(by='sharpe', ascending=False).head(5)
    print(top)

    metadata = {
        "lookahead": LOOKAHEAD,
        "train_range": [str(train["datetime"].min()), str(train["datetime"].max())],
        "test_range": [str(test["datetime"].min()), str(test["datetime"].max())],
        "features_used": top_features,
        "lgbm_params": lgbm_params,
        "catboost_params": catboost_params,
        "meta_params": meta_params
    }
    with open(f"model_metadata_{LOOKAHEAD}.json", "w") as f:
        json.dump(metadata, f, indent=2)
        
    joblib.dump(stack, f"stack_model_LOOKAHEAD_{LOOKAHEAD}_session_less.pkl")
    with open("stack_features.json", "w") as f:
        json.dump(top_features, f)

    return {
        'lookahead': LOOKAHEAD,
        'pnl': results['pnl'],
        'win_rate': results['win_rate'],
        'expectancy': results['expectancy'],
        'profit_factor': results['profit_factor'],
        'sharpe': results['sharpe'],
        'trades': results['trades'],
        'preds_lgbm': preds_lgbm,
        'preds_stack': preds_stack,
        'pres_catboost': preds_catboost,
        'results': all_results,
    }

In [ ]:
def run_lookahead(LOOKAHEAD):
    try:
        result = run_lookahead_for_session(LOOKAHEAD)
        if result is None:
            print(f"No valid run for session_less, skipping CSV.")
            return
        return result
    except Exception as e:
        print(f"⚠️ Skipping session session_less due to error: {e}")
        return

##### Running Train

In [ ]:
lookahead_values = [5]
lookahead_results = []

for val in lookahead_values:
    result = run_lookahead(val)
    lookahead_results.append(result)

# Visualize

In [ ]:
# for result in lookahead_results:
#     stack_preds = result['stack'].predict(X_test_scaled)
#     rf_preds = result['models']['rf'].predict(X_test_scaled)
#     xgb_preds = result['models']['xgb'].predict(X_test_scaled)
#     enet_preds = result['models']['elasticnet'].predict(X_test_scaled)
    
#     plt.figure(figsize=(12, 4))
#     plt.plot(rf_preds[:100], label='RF')
#     plt.plot(xgb_preds[:100], label='XGB')
#     plt.plot(enet_preds[:100], label='ElasticNet')
#     plt.plot(stack_preds[:100], label='Stack', linewidth=2)

In [ ]:
# for run in lookahead_results:
#     for r in run['results']:
#         print(r)
#         df = r['results'].copy()
#         df = df.sort_values(by='entry_time')
#         df['cumulative_pnl'] = df['pnl'].cumsum()

#         if df['cumulative_pnl'].iloc[-1] > 0 and r['sharpe'] > 10 and r['trades'] > 150 and r['win_rate'] > 0.55 and r['profit_factor'] > 1.5 and r['expectancy'] > 0.5 and r['pnl'] > 50000:
#                 plt.figure(figsize=(12, 4))
#                 plt.plot(df['entry_time'], df['cumulative_pnl'], label='Cumulative PnL', color='green')
#                 plt.title(f"PnL | Lookahead={run['lookahead']} | Sharpe={r['sharpe']:.2f}")
#                 plt.xlabel("Datetime")
#                 plt.ylabel("PnL")
#                 plt.grid(True)
#                 plt.legend()
#                 plt.tight_layout()
#                 plt.show()

In [ ]:
# Best result holder by lookahead value
best_by_lookahead = {
    5: {'win_rate': float('-inf'), 'result': None},
    15: {'win_rate': float('-inf'), 'result': None}
}

# Fill best_by_lookahead from results
for run in lookahead_results:
    lookahead = run['lookahead']
    if lookahead in best_by_lookahead:
        for r in run['results']:
            if r['win_rate'] > best_by_lookahead[lookahead]['win_rate']:
                best_by_lookahead[lookahead] = {
                    'win_rate': r['win_rate'],
                    'result': r,
                    'lookahead': lookahead
                }

# Display results nicely
for lookahead in [5]:
    best = best_by_lookahead[lookahead]
    if best['result']:
        df = best['result']['results'].copy()
        df = df.sort_values(by='entry_time')
        df['cumulative_pnl'] = df['pnl'].cumsum()

        # Set float format for readable output
        pd.options.display.float_format = '{:,.2f}'.format

        print(f"\n🏆 Best Win Rate Result for Lookahead={lookahead}")
        print(f"Win Rate: {best['win_rate']:.2%}")
        print(f"PnL: {best['result']['pnl']:.2f}")
        print(f"Trades: {best['result']['trades']}")
        print(f"Sharpe: {best['result']['sharpe']:.2f}")
        print(f"Expectancy: {best['result']['expectancy']:.2f}")
        print(f"Profit Factor: {best['result']['profit_factor']:.2f}")
        print(f"Params: {best['result']['params']}")

        print("\n🧾 All Trades from Best Win Rate Result:")
        print(df[['entry_time', 'exit_time', 'side', 'entry_price', 'exit_price',
                  'pnl', 'mfe', 'mae', 'cumulative_pnl']].to_string(index=False))

        # Plot cumulative PnL
        plt.figure(figsize=(12, 4))
        plt.plot(df['entry_time'], df['cumulative_pnl'], label='Cumulative PnL', color='blue')
        plt.title(f"Best Win Rate Run | Lookahead={lookahead} | Win Rate={best['win_rate']:.2%}")
        plt.xlabel("Datetime")
        plt.ylabel("Cumulative PnL")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print(f"No valid result found for Lookahead={lookahead}.")

In [ ]:
np.corrcoef([lookahead_results['preds_rf'], lookahead_results['preds_xgb'], lookahead_results['preds_elasticnet']])
preds_matrix = np.vstack([lookahead_results['preds_rf'], lookahead_results['preds_xgb'], lookahead_results['preds_elasticnet']])
corr_matrix = np.corrcoef(preds_matrix)

plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, xticklabels=['RF', 'XGB', 'ENet'], yticklabels=['RF', 'XGB', 'ENet'], cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Base Model Predictions")
plt.show()

# Test Model

# Sort and Plot

In [ ]:
# Predictions
# y_pred = best_lookahead.predict(X_test)
best_lookahead = max(lookahead_results, key=lambda x: max(r['pnl'] for r in x['results']))
y_pred = best_lookahead['stack'].predict(X_test_scaled)

# Confusion Matrix
labels = sorted(class_mapping)  # Make sure the order matches
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Display Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred, labels=labels, digits=2))